In [ ]:
pip install together

In [ ]:
import os
import time
import numpy as np
from dotenv import load_dotenv
from bs4 import BeautifulSoup
import re
from together import Together

# Load API key
load_dotenv("variables.env") 
together_api_key = os.getenv("TOGETHER_API_KEY")

# Set up Together AI Client
client = Together(api_key=together_api_key)

# Path to the folder containing HTML files (update this path)
html_folder = "/path/to/your/conversation_dataset"

# Function to extract word counts from <p> tags
def extract_word_counts_from_html(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        soup = BeautifulSoup(f, "html.parser")
        conversation_turns = soup.find_all("p")
        word_counts = [len(turn.get_text().split()) for turn in conversation_turns]
        return word_counts

# Aggregate word counts from specific HTML files
all_word_counts = []
for i in range(1, 41):
    html_file = os.path.join(html_folder, f"conv{i}.html")
    if os.path.exists(html_file):
        all_word_counts.extend(extract_word_counts_from_html(html_file))

# If no existing files, use default word counts
if not all_word_counts:
    all_word_counts = [3, 5, 7, 10, 12, 15, 18, 20]

# Function to generate AI-like response lengths
def generate_response_length():
    return max(np.random.choice(all_word_counts), 7)


def truncate_text(text, max_words):
    words = text.split()
    if len(words) <= max_words:
        return text 

    truncated_text = " ".join(words[:max_words])

    # Only apply re.sub() if punctuation exists
    if any(p in truncated_text for p in ".!?"):
        truncated_text = re.sub(r'[^.!?]*$', '', truncated_text).strip()
    else:
        truncated_text = text

    # If truncation removes everything, just return the original truncation
    if not truncated_text.strip():
        truncated_text = text

    return truncated_text 

In [ ]:
together_api_key

In [ ]:
# Model and parameters
model_A = "meta-llama/Llama-3.3-70B-Instruct-Turbo"
model_B = "meta-llama/Llama-3.3-70B-Instruct-Turbo" 
temperature = 0.6
temperature_regen = 1
length_convo = 22 # number of exchanges -2 (the beginning ones)

list_topics = ['fashion', 'politics', 'books', 'sports', 'general entertainment', 'music', 'science', 'technology', 'movies', 'food']


list_first_turns = ['Hello! How are you doing?', 
                 'Hey', 
                 'Hey! How is it going', 
                 'Hey, how goes it', 
                 'Yo, how’s your day been?', 
                 'What’s up?', 
                 'Sup, anything interesting going on?', 
                 'hey, anything interesting going on?', 
                 'how’ve you been?', 
                 'ciao :)', 
                 'holaaaa', 
                 'I swear today has been the longest day ever.', 
                 'Ugh, I need to vent for a sec, you got time?',
                 'Yo, I need your opinion on something real quick', 
                 'Morning!',
                 'How are the vibes today?', 
                 'Hi!'
                 'Hi there', 
                 'Hey friend', 
                 'Hey friend, how are you', 
                 'Hey my friend, how is it going']

for conv in range(33, 34):  # Generate conversations 33-50
    time.sleep(1)
    filename = f'conv{conv}.html'
    with open(filename, "w") as file_html:

        personalityA = np.random.choice(["logical and skeptical", "excitable and passionate", "sarcastic and witty", "serious and intellectual"])
        personalityB = np.random.choice(["curious and open-minded", "pragmatic and reserved", "opinionated and strong-willed", "friendly and cheerful"])

        # Assign A's age (as an integer)
        ageA = int(np.random.choice([18, 25, 32, 40, 50, 60, 70]))

        # Assign B's age within ±5 years of A
        ageB = np.random.choice([max(18, ageA - 5), ageA, min(70, ageA + 5)])  

        conversation_topic = np.random.choice(list_topics)

        # System Prompts (Each AI only sees their own role)
        system_prompt_A = f"""
        A is {ageA} years old and is {personalityA}. A is having a natural conversation with a friend about {conversation_topic}. 
        A speaks naturally, responds informally, and does NOT announce persona. 
        Do NOT restart the conversation or summarize past dialogue.
        A sometimes challenges opinions or plays devil's advocate.
        Do NOT use asterisks (`*`) to indicate actions or commentary. Keep answers under 20 tokens. Use abbreviations and punctuations as is representative of your age.
        Do NOT use emojis
        ONLY respond as A.
        """

        system_prompt_B = f"""
        B is {ageB} years old and is {personalityB}. B is having a natural conversation with a friend about {conversation_topic}. 
        B speaks naturally, responds informally, and does NOT announce persona. 
        B is opinionated and occasionally disagrees, but in a friendly way.
        Do NOT restart the conversation or summarize past dialogue.
        Do NOT use asterisks (`*`) to indicate actions or commentary. Keep answers under 20 tokens. Use abbreviations and punctuations as is representative of your age.
        Do NOT use emojis
        ONLY respond as B.
        """

        # Explicitly Start the First Turn
        first_turn = np.random.choice(list_first_turns)
        conversation_history = [{"role": "user", "content": first_turn}]
        new_line = f"<p> A: {first_turn} </p>\n"
        file_html.write(new_line)

        # Generate B's First Response Immediately
        response = client.chat.completions.create(
            model=model_B,
            messages=[
                {"role": "system", "content": system_prompt_B},
                {"role": "user", "content": first_turn}
            ],
            max_tokens=80,
            temperature=temperature,
            top_p=1.0,
            stop=["A:", "B:"]
        )

        if response.choices[0].message.content:
            first_response = response.choices[0].message.content.strip()
            conversation_history.append({"role": "user", "content": f"B: {first_response}"})  
            new_line = f"<p> B: {first_response} </p>\n"
            file_html.write(new_line)

        # Now continue the conversation normally
        for i in range(length_convo):
            response_length = generate_response_length()
            temperature_step = temperature + ((temperature_regen - temperature) / length_convo) * i

            if i % 2 == 0:
                speaker = "A"
                system_prompt = system_prompt_A
                model = model_A
            else:
                speaker = "B"
                system_prompt = system_prompt_B
                model = model_B

            # Prepare messages for Together AI
            messages = [{"role": "system", "content": system_prompt}]
            
            # Add conversation history
            for msg in conversation_history:
                messages.append({"role": "user", "content": msg["content"]})

            response = None
            attempts = 0
            max_attempts = 3

            while attempts < max_attempts:
                response = client.chat.completions.create(
                    model=model,
                    messages=messages,
                    max_tokens=80,
                    temperature=temperature_step,
                    top_p=1.0,
                    stop=["A:", "B:"]
                )
                    
                    # Check if we got actual content
                if response.choices[0].message.content and response.choices[0].message.content.strip():
                        break  # We got a good response, exit the loop
                    
                attempts += 1

            if not response.choices[0].message.content or not response.choices[0].message.content.strip():
                string = "..."  # Or any other fallback you prefer
            else:
                string = response.choices[0].message.content.strip()

            # response = client.chat.completions.create(
            #     model=model,
            #     messages=messages,
            #     max_tokens=80,
            #     temperature=temperature_step,
            #     top_p=1.0,
            #     stop=["A:", "B:"]
            # )

            if response.choices[0].message.content:
                string = response.choices[0].message.content.strip()
                
                # Ensure minimum word count
                while len(string.split()) < 2: 
                    response = client.chat.completions.create(
                        model=model,
                        messages=messages,
                        max_tokens=80,
                        temperature=temperature_step,
                        top_p=1.0,
                        stop=[]
                    )
                    string = response.choices[0].message.content.strip()
            else:
                string = "..."

            string = truncate_text(string, response_length)

            # Store Response for Next AI Agent
            conversation_history.append({"role": "user", "content": f"{speaker}: {string}"})

            # Write Proper Alternating Turns
            new_line = f"<p> {speaker}: {string} </p>\n"
            file_html.write(new_line)

        print(f"Generated conversation {conv}")
        print("\n".join([msg["content"] for msg in conversation_history]))
        print("="*50)

In [ ]:
import anthropic
import os
from dotenv import load_dotenv

load_dotenv("variables.env")

# Retrieve the API key
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")


client = anthropic.Anthropic(api_key=anthropic_api_key)
print(client)



In [ ]:
print(anthropic_api_key)

In [ ]:
models = client.models.list()
print(models)

In [ ]:
response = client.messages.create(
    model= "claude-3-5-sonnet-20241022" #"claude-3-opus-20240229",  # Adjust based on the model version you have access to
    max_tokens=1024,
    messages=[
        {"role": "user", "content": "Hello, how are you?"}
    ]
)

print(response.content)


In [ ]:
response

In [ ]:
import anthropic
model = "claude-3-5-sonnet-20241022"

client = anthropic.Anthropic(api_key=anthropic_api_key)

def get_completion(prompt: str, system_prompt=""):
    message = client.messages.create(
        model=model,
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        messages=[
          {"role": "user", "content": prompt}
        ]
    )
    return message.content[0].text

In [ ]:
# Prompt
prompt = "Hello, Claude!"

# Get Claude's response
print(get_completion(prompt))

In [ ]:

# System prompt
SYSTEM_PROMPT = "Your answer should always be a series of critical thinking questions that further the conversation (do not provide answers to your questions). Do not actually answer the user question."

# Prompt
PROMPT = "Why is the sky blue?"

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

In [ ]:
def get_completion(prompt: str, system_prompt="", prefill=""):
    message = client.messages.create(
        model=model,
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        messages=[
          {"role": "user", "content": prompt},
          {"role": "assistant", "content": prefill}
        ]
    )
    return message.content[0].text

In [ ]:
# Prompt
PROMPT = "Will Santa bring me presents on Christmas?"

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:


# Prompt
PROMPT = """Please complete the conversation by writing the next line, speaking as "A".
Q: Is the tooth fairy real?
A: Of course, sweetie. Wrap up your tooth and put it under your pillow tonight. There might be something waiting for you in the morning.
Q: Will Santa bring me presents on Christmas?"""

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
import random

def casual_text_formatting(text):
    """Randomly convert responses to lowercase to mimic casual texting."""
    if random.random() < 0.5:  # 50% chance of being lowercase
        return text.lower()
    return text

In [ ]:
import anthropic
import os
import time
import numpy as np
from dotenv import load_dotenv
from bs4 import BeautifulSoup
import re


# Load API key
load_dotenv()
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")

# Set up Anthropic Client
client = anthropic.Anthropic(api_key=anthropic_api_key)

# Path to the folder containing HTML files
html_folder = "/Users/elisapavarino/Documents/Work_Directory/Kreiman_Lab/Turing Test Paper/TuringGithub/conversation/conversation_task/conversation_dataset"

# Function to extract word counts from <p> tags
def extract_word_counts_from_html(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        soup = BeautifulSoup(f, "html.parser")
        conversation_turns = soup.find_all("p")
        word_counts = [len(turn.get_text().split()) for turn in conversation_turns]
        return word_counts

# Aggregate word counts from specific HTML files
all_word_counts = []
for i in range(1, 41):
    html_file = os.path.join(html_folder, f"conv{i}.html")
    if os.path.exists(html_file):
        all_word_counts.extend(extract_word_counts_from_html(html_file))

# Function to generate AI-like response lengths
def generate_response_length():
    return max(np.random.choice(all_word_counts), 7)

# Function to truncate text at a natural stopping point
def truncate_text(text, max_words):
    words = text.split()
    if len(words) <= max_words:
        return text
    truncated_text = " ".join(words[:max_words])
    return re.sub(r'[^.!?]*$', '', truncated_text).strip()

# Model and parameters
model = model #"claude-3-opus-20240229"  # Adjust model based on your access
temperature = 0.6
temperature_regen = 1

list_names1 = ['John', 'Alice', 'Mark', 'Andrew', 'Julia', 'Georgia', 'Juliet', 'Lily', 'Olivia', 'Emmett', 'Miles', 'Oscar', 'William']
list_names2 = ['Frank', 'Tom', 'Elizabeth', 'Gabriel', 'Paul', 'Alfie', 'Christina', 'Edward', 'Ella', 'Oliver', 'Karen', 'Isabel', 'Brad']
list_topics = ['fashion', 'politics', 'books', 'sports', 'general entertainment', 'music', 'science', 'technology', 'movies', 'food']

for conv in range(24, 25):
    time.sleep(1)
    filename = f'conv{conv}.html'
    with open(filename, "w") as file_html:
        friend1 = np.random.choice(list_names1)
        friend2 = np.random.choice(list_names2)
        topic = np.random.choice(list_topics)

        personality1 = np.random.choice(["logical and skeptical", "excitable and passionate", "sarcastic and witty", "serious and intellectual"])
        personality2 = np.random.choice(["curious and open-minded", "pragmatic and reserved", "opinionated and strong-willed", "friendly and cheerful"])

        # Anthropic handles system prompts separately
        system_prompt = f"{friend2} is {personality2} and is having a natural conversation with a friend. Avoid excessive agreement and looping. Keep answers under 20 tokens. If a topic has been discussed for too long, transition to a new subject smoothly. Be spare with punctuation. You can insert one spelling mistake. Keep answers short, text-like, and very informal. DO NOT always end with questions. No skipped turns."  #Instead of repeating excitement, introduce new questions or challenge each other’s views. 

        user_prompt = f"{friend1} is {personality1} and is having a natural conversation with a friend. They discuss {topic}, but they don’t always agree. Each person has their own perspective, and they challenge each other at times. They avoid excessive agreement and mutual praise. Be spare with punctuation. You can insert one spelling mistake. They ensure to change topics when the discussion slows down. Keep answers short, text-like, and very informal. **Do not ask a question at the end of your response. Conclude with a statement, reflection, or suggestion.**\n" + "A: Hi!\n:"

        for i in range(12):
            num_words = 0
            cnt = 0
            while num_words < 5 and cnt < 6:
                response_length = generate_response_length()
                #print(f"Debug: response_length = {response_length}")

                
                temperature_step = temperature + ((temperature_regen - temperature) / 5) * cnt

                response = client.messages.create(
                    model=model,
                    max_tokens=80,
                    system=system_prompt,  # System message goes here in Anthropic API
                    messages=[{"role": "user", "content": user_prompt}],
                    temperature=temperature_step,
                    top_p=1.0,
                    stop_sequences= [] #[f"{friend1}:", f"{friend2}:"]  # Equivalent to stop in OpenAI
                )
                
                string = " ".join([block.text for block in response.content]).strip() if response.content else ""
                string = truncate_text(string, response_length)
                string = casual_text_formatting(string)
                num_words = len(string.split())
                cnt += 1
            
            speaker = f"\n{friend1}:"
            new_line = f"<p> A: {string} </p>\n"
            file_html.write(new_line)
            user_prompt += string + speaker

            num_words = 0
            cnt = 0
            
            while num_words < 5 and cnt < 6:
                response_length = generate_response_length()
                #print(f"Debug: response_length = {response_length}")

                
                temperature_step = temperature + ((temperature_regen - temperature) / 5) * cnt

                response = client.messages.create(
                    model=model,
                    max_tokens=80,
                    system=system_prompt,
                    messages=[{"role": "user", "content": user_prompt}],
                    temperature=temperature_step,
                    top_p=1.0,
                    stop_sequences= [] #[f"{friend1}:", f"{friend2}:"]
                )

                string = " ".join([block.text for block in response.content]).strip() if response.content else ""
                string = truncate_text(string, response_length)
                string = casual_text_formatting(string)
                num_words = len(string.split())
                cnt += 1
            
            speaker = f"\n{friend2}:"
            new_line = f"<p> B: {string} </p>\n"
            file_html.write(new_line)
            user_prompt += string + speaker
        
        print(user_prompt)


# Current version

In [ ]:
import anthropic
import os
import time
import numpy as np
from dotenv import load_dotenv
from bs4 import BeautifulSoup
import re


# Load API key
load_dotenv("variables.env") 
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")

# Set up Anthropic Client
client = anthropic.Anthropic(api_key=anthropic_api_key)

# Path to the folder containing HTML files
html_folder = "/Users/elisapavarino/Documents/Work_Directory/Kreiman_Lab/Turing Test Paper/TuringGithub/conversation/conversation_task/conversation_dataset"

# Function to extract word counts from <p> tags
def extract_word_counts_from_html(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        soup = BeautifulSoup(f, "html.parser")
        conversation_turns = soup.find_all("p")
        word_counts = [len(turn.get_text().split()) for turn in conversation_turns]
        return word_counts

# Aggregate word counts from specific HTML files
all_word_counts = []
for i in range(1, 41):
    html_file = os.path.join(html_folder, f"conv{i}.html")
    if os.path.exists(html_file):
        all_word_counts.extend(extract_word_counts_from_html(html_file))

# Function to generate AI-like response lengths
def generate_response_length():
    return max(np.random.choice(all_word_counts), 7)

def truncate_text(text, max_words):
    words = text.split()
    if len(words) <= max_words:
        return text if text.strip() else "..."  # Ensure we never return empty text

    truncated_text = " ".join(words[:max_words])

    # Only apply re.sub() if punctuation exists
    if any(p in truncated_text for p in ".!?"):
        truncated_text = re.sub(r'[^.!?]*$', '', truncated_text).strip()
    else:
        truncated_text = text

    # If truncation removes everything, just return the original truncation
    if not truncated_text.strip():
        truncated_text = text

    return truncated_text 

# Model and parameters
model_A = "claude-3-5-sonnet-20241022"
model_B = "claude-3-5-sonnet-20241022" 
temperature = 0.6
temperature_regen = 1
length_convo = 22 # number of exchanges -2 (the beginning ones)

list_topics = ['fashion', 'politics', 'books', 'sports', 'general entertainment', 'music', 'science', 'technology', 'movies', 'food']

list_first_turns = ['Hello! How are you doing?', 
                 'Hey', 
                 'Hey! How is it going', 
                 'Hey, how goes it', 
                 'Yo, how’s your day been?', 
                 'What’s up?', 
                 'Sup, anything interesting going on?', 
                 'hey, anything interesting going on?', 
                 'how’ve you been?', 
                 'ciao :)', 
                 'holaaaa', 
                 'I swear today has been the longest day ever.', 
                 'Ugh, I need to vent for a sec, you got time?',
                 'Yo, I need your opinion on something real quick', 
                 'Morning!',
                 'How are the vibes today?', 
                 'Hi!'
                 'Hi there', 
                 'Hey friend', 
                 'Hey friend, how are you', 
                 'Hey my friend, how is it going']

for conv in range(31, 33):
    time.sleep(1)
    filename = f'conv{conv}.html'
    with open(filename, "w") as file_html:

        personalityA = np.random.choice(["logical and skeptical", "excitable and passionate", "sarcastic and witty", "serious and intellectual"])
        personalityB = np.random.choice(["curious and open-minded", "pragmatic and reserved", "opinionated and strong-willed", "friendly and cheerful"])

         # Assign A's age (as an integer)
        ageA = int(np.random.choice([18, 25, 32, 40, 50, 60, 70]))

        # Assign B's age within ±5 years of A
        ageB = np.random.choice([max(18, ageA - 5), ageA, min(70, ageA + 5)])  



        conversation_topic = np.random.choice(list_topics)

        # System Prompts (Each AI only sees their own role)
        system_prompt_A = f"""
        A is {ageA} years old and is {personalityA}. A is having a natural conversation with a friend about {conversation_topic}. 
        A speaks naturally, responds informally, and does NOT announce persona. 
        Do NOT restart the conversation or summarize past dialogue.
        A sometimes challenges opinions or plays devil’s advocate.
        Do NOT use asterisks (`*`) to indicate actions or commentary. Keep answers under 20 tokens. Use abbreviations and punctuations as is representative of your age.
        Do NOT use emojis
        ONLY respond as A.
        """

        system_prompt_B = f"""
        B is {ageB} years old and is {personalityB}. B is having a natural conversation with a friend about {conversation_topic}. 
        B speaks naturally, responds informally, and does NOT announce persona. 
        B is opinionated and occasionally disagrees, but in a friendly way.
        Do NOT restart the conversation or summarize past dialogue.
        Do NOT use asterisks (`*`) to indicate actions or commentary. Keep answers under 20 tokens. Use abbreviations and punctuations as is representative of your age.
        Do NOT use emojis
        ONLY respond as B.
        """

        # Explicitly Start the First Turn
        first_turn = np.random.choice(list_first_turns) #" Hey there! What's up?"
        conversation_history = [{"role": "user", "content": first_turn}]
        new_line = f"<p> A: {first_turn} </p>\n"
        file_html.write(new_line)


        # Generate B's First Response Immediately
        response = client.messages.create(
            model=model_B,
            max_tokens=80,
            system=system_prompt_B,  
            messages=conversation_history,  
            temperature=temperature,
            top_p=1.0,
            stop_sequences=["A:", "B:"]
        )

        if response.content:
            first_response = " ".join([block.text for block in response.content]).strip()
            conversation_history.append({"role": "user", "content": f"B: {first_response}"})  
            new_line = f"<p> B: {first_response} </p>\n"
            file_html.write(new_line)

        # Now continue the conversation normally
        for i in range(length_convo):  # 11 more turns each after the first exchange
            response_length = generate_response_length()
            temperature_step = temperature + ((temperature_regen - temperature) / length_convo) * i

            if i % 2 == 0:
                speaker = "A"
                system_prompt = system_prompt_A
                model = model_A
            else:
                speaker = "B"
                system_prompt = system_prompt_B
                model = model_B

            response = client.messages.create(
                model=model,
                max_tokens=80,
                system=system_prompt,  
                messages=conversation_history,  
                temperature=temperature_step,
                top_p=1.0,
                stop_sequences=["A:", "B:"] 
            )

            if response.content:
                string = " ".join([block.text for block in response.content]).strip()
                while len(string.split()) < 2: 
                    response = client.messages.create(
                        model=model,
                        max_tokens=80,
                        system=system_prompt,
                        messages=conversation_history,  
                        temperature=temperature,
                        top_p=1.0,
                        stop_sequences=[]
                    )
                    string = " ".join([block.text for block in response.content]).strip()

            else:
                string = "..."

            string = truncate_text(string, response_length)

            # Store Response for Next AI Agent
            conversation_history.append({"role": "user", "content": f"{speaker}: {string}"})

            # Write Proper Alternating Turns
            new_line = f"<p> {speaker}: {string} </p>\n"
            file_html.write(new_line)

        print("\n".join([msg["content"] for msg in conversation_history]))  


In [ ]:
import anthropic
import os
import time
import numpy as np
from dotenv import load_dotenv
from bs4 import BeautifulSoup
import re


# Load API key
load_dotenv()
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")

# Set up Anthropic Client
client = anthropic.Anthropic(api_key=anthropic_api_key)

# Path to the folder containing HTML files
html_folder = "/Users/elisapavarino/Documents/Work_Directory/Kreiman_Lab/Turing Test Paper/TuringGithub/conversation/conversation_task/conversation_dataset"

# Function to extract word counts from <p> tags
def extract_word_counts_from_html(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        soup = BeautifulSoup(f, "html.parser")
        conversation_turns = soup.find_all("p")
        word_counts = [len(turn.get_text().split()) for turn in conversation_turns]
        return word_counts

# Aggregate word counts from specific HTML files
all_word_counts = []
for i in range(1, 41):
    html_file = os.path.join(html_folder, f"conv{i}.html")
    if os.path.exists(html_file):
        all_word_counts.extend(extract_word_counts_from_html(html_file))

# Function to generate AI-like response lengths
def generate_response_length():
    return max(np.random.choice(all_word_counts), 7)

def truncate_text(text, max_words):
    
    words = text.split()
    if len(words) <= max_words:
        return text if text.strip() else "..."  # Ensure we never return empty text

    truncated_text = " ".join(words[:max_words])

    # Only apply re.sub() if punctuation exists
    if any(p in truncated_text for p in ".!?"):
        truncated_text = re.sub(r'[^.!?]*$', '', truncated_text).strip()

    # If truncation removes everything, just return the original truncation
    if not truncated_text.strip():
        truncated_text = text

    return truncated_text 


# Model and parameters
model = model #"claude-3-opus-20240229"  # Adjust model based on your access
temperature = 0.6
temperature_regen = 1

#list_names1 = ['John', 'Alice', 'Mark', 'Andrew', 'Julia', 'Georgia', 'Juliet', 'Lily', 'Olivia', 'Emmett', 'Miles', 'Oscar', 'William']
#list_names2 = ['Frank', 'Tom', 'Elizabeth', 'Gabriel', 'Paul', 'Alfie', 'Christina', 'Edward', 'Ella', 'Oliver', 'Karen', 'Isabel', 'Brad']
list_topics = ['fashion', 'politics', 'books', 'sports', 'general entertainment', 'music', 'science', 'technology', 'movies', 'food']

for conv in range(27, 30):
    time.sleep(1)
    filename = f'conv{conv}.html'
    with open(filename, "w") as file_html:
        #friend1 = np.random.choice(list_names1)
        #friend2 = np.random.choice(list_names2)
        topic = np.random.choice(list_topics)

        # System Prompts (Each AI only sees their own role)
        system_prompt_A = f"""
        A is {personalityA} and is having a natural conversation with a friend. 
        A speaks naturally, responds informally, and does NOT announce persona. 
        Do NOT restart the conversation or summarize past dialogue.
        Do NOT use asterisks (`*`) to indicate actions or commentary. Keep answers under 20 tokens. Be sparing with punctuation. 
        ONLY respond as A.
        """

        system_prompt_B = f"""
        B is {personalityB} and is having a natural conversation with a friend. 
        B speaks naturally, responds informally, and does NOT announce persona. 
        Do NOT restart the conversation or summarize past dialogue.
        Do NOT use asterisks (`*`) to indicate actions or commentary. Keep answers under 20 tokens. Be sparing with punctuation. 
        ONLY respond as B.
        """

        # Start conversation with an opening line from A
        conversation_history = [{"role": "user", "content": "A: Hey there! What's up?"}]

        for i in range(24):  # 12 turns each
            response_length = generate_response_length()
            temperature_step = temperature + ((temperature_regen - temperature) / 5) * i

            ### **🛠 FIX: Separate Prompts for A and B**
            if i % 2 == 0:
                speaker = "A"
                system_prompt = system_prompt_A
                model = model_A
            else:
                speaker = "B"
                system_prompt = system_prompt_B
                model = model_B

            response = client.messages.create(
                model=model,
                max_tokens=80,
                system=system_prompt,  
                messages=conversation_history,  
                temperature=temperature,
                top_p=1.0,
                stop_sequences= ["A:", "B:"] 
            )

            if response.content:
                string = " ".join([block.text for block in response.content]).strip()
                while len(string.split()) < 2: 
                    response = client.messages.create(
                        model=model,
                        max_tokens=80,
                        system=system_prompt,
                        messages=conversation_history,  
                        temperature=temperature,
                        top_p=1.0,
                        stop_sequences=[]
                    )
                    string = " ".join([block.text for block in response.content]).strip()

            else:
                string = "..."

            string = truncate_text(string, response_length)

            ### **🛠 FIX: Store Response for Next AI Agent**
            conversation_history.append({"role": "user", "content": f"{speaker}: {string}"})  # ✅ Save each AI’s response

            ### **🛠 FIX: Write Proper Alternating Turns**
            new_line = f"<p> {speaker}: {string} </p>\n"
            file_html.write(new_line)

        
        print(user_prompt)


In [ ]:
import anthropic
import os
import time
import numpy as np
from dotenv import load_dotenv
from bs4 import BeautifulSoup
import re


# Load API key
load_dotenv()
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")

# Set up Anthropic Client
client = anthropic.Anthropic(api_key=anthropic_api_key)

# Path to the folder containing HTML files
html_folder = "/Users/elisapavarino/Documents/Work_Directory/Kreiman_Lab/Turing Test Paper/TuringGithub/conversation/conversation_task/conversation_dataset"

# Function to extract word counts from <p> tags
def extract_word_counts_from_html(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        soup = BeautifulSoup(f, "html.parser")
        conversation_turns = soup.find_all("p")
        word_counts = [len(turn.get_text().split()) for turn in conversation_turns]
        return word_counts

# Aggregate word counts from specific HTML files
all_word_counts = []
for i in range(1, 41):
    html_file = os.path.join(html_folder, f"conv{i}.html")
    if os.path.exists(html_file):
        all_word_counts.extend(extract_word_counts_from_html(html_file))

# Function to generate AI-like response lengths
def generate_response_length():
    return max(np.random.choice(all_word_counts), 7)

def truncate_text(text, max_words):
    
    words = text.split()
    if len(words) <= max_words:
        return text if text.strip() else "..."  # Ensure we never return empty text

    truncated_text = " ".join(words[:max_words])

    # Only apply re.sub() if punctuation exists
    if any(p in truncated_text for p in ".!?"):
        truncated_text = re.sub(r'[^.!?]*$', '', truncated_text).strip()

    # If truncation removes everything, just return the original truncation
    if not truncated_text.strip():
        truncated_text = text

    return truncated_text if truncated_text.strip() else "..."  # Ensure something is returned


# Model and parameters
model = model #"claude-3-opus-20240229"  # Adjust model based on your access
temperature = 0.6
temperature_regen = 1

#list_names1 = ['John', 'Alice', 'Mark', 'Andrew', 'Julia', 'Georgia', 'Juliet', 'Lily', 'Olivia', 'Emmett', 'Miles', 'Oscar', 'William']
#list_names2 = ['Frank', 'Tom', 'Elizabeth', 'Gabriel', 'Paul', 'Alfie', 'Christina', 'Edward', 'Ella', 'Oliver', 'Karen', 'Isabel', 'Brad']
list_topics = ['fashion', 'politics', 'books', 'sports', 'general entertainment', 'music', 'science', 'technology', 'movies', 'food']

for conv in range(25, 26):
    time.sleep(1)
    filename = f'conv{conv}.html'
    with open(filename, "w") as file_html:
        #friend1 = np.random.choice(list_names1)
        #friend2 = np.random.choice(list_names2)
        topic = np.random.choice(list_topics)

        personality1 = np.random.choice(["logical and skeptical", "excitable and passionate", "sarcastic and witty", "serious and intellectual"])
        personality2 = np.random.choice(["curious and open-minded", "pragmatic and reserved", "opinionated and strong-willed", "friendly and cheerful"])

        # Anthropic handles system prompts separately
        system_prompt = f"A is {personality2} and is having a natural conversation with a friend. Avoid excessive agreement and looping. Keep answers under 20 tokens. If a topic has been discussed for too long, transition to a new subject smoothly. Be spare with punctuation. You can insert one spelling mistake. Keep answers short, text-like, and very informal. DO NOT always end with questions. No skipped turns."  #Instead of repeating excitement, introduce new questions or challenge each other’s views. 

        user_prompt = f"B is {personality1} and is having a natural conversation with a friend. They discuss {topic}, but they don’t always agree. Each person has their own perspective, and they challenge each other at times. They avoid excessive agreement and mutual praise. Be spare with punctuation. You can insert one spelling mistake. They ensure to change topics when the discussion slows down. Keep answers short, text-like, and very informal. **Do not ask a question at the end of your response. Conclude with a statement, reflection, or suggestion.**\n" + "A: Hi!\n:"

        for i in range(12):
            num_words = 0
            cnt = 0
            while num_words < 5 and cnt < 6:
                response_length = generate_response_length()
                #print(f"Debug: response_length = {response_length}")

                
                temperature_step = temperature + ((temperature_regen - temperature) / 5) * cnt

                response = client.messages.create(
                    model=model,
                    max_tokens=80,
                    system=system_prompt,  # System message goes here in Anthropic API
                    messages=[{"role": "user", "content": user_prompt}],
                    temperature=temperature_step,
                    top_p=1.0,
                    stop_sequences= [] #[f"{friend1}:", f"{friend2}:"]  # Equivalent to stop in OpenAI
                )
                
                if response.content:
                    string = " ".join([block.text for block in response.content]).strip()
                    
                    #  **Keep regenerating if response is empty**
                    while len(string.split()) < 2:  # Accepts anything longer than 1 word
                        response = client.messages.create(
                            model=model,
                            max_tokens=80,
                            system=system_prompt,
                            messages=[{"role": "user", "content": user_prompt}],
                            temperature=temperature_step,
                            top_p=1.0,
                            stop_sequences=[]
                        )
                        string = " ".join([block.text for block in response.content]).strip()
                    
                else:
                    string = "..."  #  Fallback if somehow we still get no response


                string = truncate_text(string, response_length)
                string = casual_text_formatting(string)
                num_words = len(string.split())
                cnt += 1
            
            speaker = f"\n{friend1}:"
            new_line = f"<p> A: {string} </p>\n"
            file_html.write(new_line)
            user_prompt += string + "A: "

            num_words = 0
            cnt = 0
            
            while num_words < 5 and cnt < 6:
                response_length = generate_response_length()
                #print(f"Debug: response_length = {response_length}")

                
                temperature_step = temperature + ((temperature_regen - temperature) / 5) * cnt

                response = client.messages.create(
                    model=model,
                    max_tokens=80, #min(60, int(response_length * 1.3)),
                    system=system_prompt,
                    messages=[{"role": "user", "content": user_prompt}],
                    temperature=temperature_step,
                    top_p=1.0,
                    stop_sequences= [] #[f"{friend1}:", f"{friend2}:"]
                )

                if response.content:
                    string = " ".join([block.text for block in response.content]).strip()
                    
                    # **Keep regenerating if response is empty**
                    while len(string.split()) < 2:  # Accepts anything longer than 1 word
                        response = client.messages.create(
                            model=model,
                            max_tokens= 80, #min(60, int(response_length * 1.3)),
                            system=system_prompt,
                            messages=[{"role": "user", "content": user_prompt}],
                            temperature=temperature_step,
                            top_p=1.0,
                            stop_sequences=[]
                        )
                        string = " ".join([block.text for block in response.content]).strip()
                    
                else:
                    string = "..."  # Fallback if somehow we still get no response


                string = truncate_text(string, response_length)
                string = casual_text_formatting(string)
                num_words = len(string.split())
                cnt += 1
            
            speaker = f"\n{friend2}:"
            new_line = f"<p> B: {string} </p>\n"
            file_html.write(new_line)
            user_prompt += string + "B: "
        
        print(user_prompt)


In [ ]:
import anthropic
import os
import time
import numpy as np
from dotenv import load_dotenv
from bs4 import BeautifulSoup
import re

# Load API key
load_dotenv()
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")

# Set up Anthropic Client
client = anthropic.Anthropic(api_key=anthropic_api_key)

# Function to generate AI-like response lengths
def generate_response_length():
    return np.random.randint(3, 20)  # Ensure at least 3 words

# Function to truncate text at a natural stopping point
def truncate_text(text, max_words):
    words = text.split()
    if len(words) <= max_words:
        return text
    truncated_text = " ".join(words[:max_words])
    return re.sub(r'[^.!?]*$', '', truncated_text).strip()

# Model and parameters
model_A = "claude-3-opus-20240229"  # AI Agent A
model_B = "claude-3-5-sonnet-20241022"  # AI Agent B (or use Opus for both)
temperature = 0.6
temperature_regen = 1

list_topics = ['fashion', 'politics', 'books', 'sports', 'general entertainment', 'music', 'science', 'technology', 'movies', 'food']

for conv in range(23, 24):
    time.sleep(1)
    filename = f'conv{conv}.html'
    with open(filename, "w") as file_html:
        topic = np.random.choice(list_topics)

        personalityA = np.random.choice(["logical and skeptical", "excitable and passionate", "sarcastic and witty", "serious and intellectual"])
        personalityB = np.random.choice(["curious and open-minded", "pragmatic and reserved", "opinionated and strong-willed", "friendly and cheerful"])

        ### **🛠 FIX: A and B Get Their Own System Prompts**
        system_prompt_A = f"A is {personalityA} and is having a natural conversation with a friend. A speaks naturally, responds informally, and keeps responses short. Do not use emojis or meta-analysis."
        system_prompt_B = f"B is {personalityB} and is having a natural conversation with a friend. B speaks naturally, responds informally, and keeps responses short. Do not use emojis or meta-analysis."

        ### **🛠 FIX: Store the Full Conversation Properly**
        conversation_history = [
            {"role": "user", "content": "A: Hey there! What's up?"}
        ]

        for i in range(24):  # 12 turns each
            response_length = generate_response_length()
            temperature_step = temperature + ((temperature_regen - temperature) / 5) * i

            ### **🛠 FIX: Alternate API Calls Properly**
            if i % 2 == 0:
                speaker = "A"
                system_prompt = system_prompt_A
                model = model_A
                last_message = conversation_history[-1]["content"]  # Get the last response
            else:
                speaker = "B"
                system_prompt = system_prompt_B
                model = model_B
                last_message = conversation_history[-1]["content"]  # A’s last response becomes B’s input

            response = client.messages.create(
                model=model,
                max_tokens=80,
                system=system_prompt,
                messages=[{"role": "user", "content": last_message}],  # ✅ Feed previous AI response
                temperature=temperature,
                top_p=1.0,
                stop_sequences=[]
            )

            if response.content:
                print(f"DEBUG: {speaker} Response: {response.content}")
                string = " ".join([block.text for block in response.content]).strip()
                if len(string.split()) < 3:  # Ensure at least 3 words
                    response = client.messages.create(
                        model=model,
                        max_tokens=80,
                        system=system_prompt,
                        messages=[{"role": "user", "content": last_message}],  # ✅ Feed previous AI response
                        temperature=temperature,
                        top_p=1.0,
                        stop_sequences=[]
                    )
                    string = " ".join([block.text for block in response.content]).strip() if response.content else "(No response)"
            else:
                string = "(No response)"

            string = truncate_text(string, response_length)

            ### **🛠 FIX: Store Response for Next AI Agent**
            conversation_history.append({"role": "user", "content": f"{speaker}: {string}"})  # ✅ Save each AI’s response

            ### **🛠 FIX: Write Proper Alternating Turns**
            new_line = f"<p> {speaker}: {string} </p>\n"
            file_html.write(new_line)

        print("\n".join([msg["content"] for msg in conversation_history]))  # ✅ Print full conversation history


In [ ]:
import anthropic
import os
import time
import numpy as np
from dotenv import load_dotenv
from bs4 import BeautifulSoup
import re

# Load API key
load_dotenv()
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")

# Set up Anthropic Client
client = anthropic.Anthropic(api_key=anthropic_api_key)

# Function to generate AI-like response lengths
def generate_response_length():
    return np.random.randint(3, 20)  # Ensure at least 3 words

# Function to truncate text at a natural stopping point
def truncate_text(text, max_words):
    words = text.split()
    if len(words) <= max_words:
        return text
    truncated_text = " ".join(words[:max_words])
    return re.sub(r'[^.!?]*$', '', truncated_text).strip()

# Model and parameters
model_A = "claude-3-opus-20240229"  # AI Agent A
model_B = "claude-3-5-sonnet-20241022"  # AI Agent B (or use Opus for both)
temperature = 0.6
temperature_regen = 1

list_topics = ['fashion', 'politics', 'books', 'sports', 'general entertainment', 'music', 'science', 'technology', 'movies', 'food']

for conv in range(23, 24):
    time.sleep(1)
    filename = f'conv{conv}.html'
    with open(filename, "w") as file_html:
        topic = np.random.choice(list_topics)

        personalityA = np.random.choice(["logical and skeptical", "excitable and passionate", "sarcastic and witty", "serious and intellectual"])
        personalityB = np.random.choice(["curious and open-minded", "pragmatic and reserved", "opinionated and strong-willed", "friendly and cheerful"])

        ### **🛠 FIX: A and B Get Their Own System Prompts**
        system_prompt_A = f"A is {personalityA} and is having a natural conversation with a friend. A speaks naturally, responds informally, and keeps responses short. Do not use emojis or meta-analysis."
        system_prompt_B = f"B is {personalityB} and is having a natural conversation with a friend. B speaks naturally, responds informally, and keeps responses short. Do not use emojis or meta-analysis."

        ### **🛠 FIX: Store the Full Conversation Properly**
        conversation_history = [
            {"role": "user", "content": "A: Hey there! What's up?"}
        ]

        for i in range(24):  # 12 turns each
            response_length = generate_response_length()
            temperature_step = temperature + ((temperature_regen - temperature) / 5) * i

            ### **🛠 FIX: Alternate API Calls Properly**
            if i % 2 == 0:
                speaker = "A"
                system_prompt = system_prompt_A
                model = model_A
                last_message = conversation_history[-1]["content"]  # Get the last response
            else:
                speaker = "B"
                system_prompt = system_prompt_B
                model = model_B
                last_message = conversation_history[-1]["content"]  # A’s last response becomes B’s input

            response = client.messages.create(
                model=model,
                max_tokens=80,
                system=system_prompt,
                messages=[{"role": "user", "content": last_message}],  # ✅ Feed previous AI response
                temperature=temperature,
                top_p=1.0,
                stop_sequences=[]
            )

            if response.content:
                print(f"DEBUG: {speaker} Response: {response.content}")
                string = " ".join([block.text for block in response.content]).strip()
                if len(string.split()) < 3:  # Ensure at least 3 words
                    response = client.messages.create(
                        model=model,
                        max_tokens=80,
                        system=system_prompt,
                        messages=[{"role": "user", "content": last_message}],  # ✅ Feed previous AI response
                        temperature=temperature,
                        top_p=1.0,
                        stop_sequences=[]
                    )
                    string = " ".join([block.text for block in response.content]).strip() if response.content else "(No response)"
            else:
                string = "(No response)"

            string = truncate_text(string, response_length)

            ### **🛠 FIX: Store Response for Next AI Agent**
            conversation_history.append({"role": "user", "content": f"{speaker}: {string}"})  # ✅ Save each AI’s response

            ### **🛠 FIX: Write Proper Alternating Turns**
            new_line = f"<p> {speaker}: {string} </p>\n"
            file_html.write(new_line)

        print("\n".join([msg["content"] for msg in conversation_history]))  # ✅ Print full conversation history
